In [ ]:
import pyopia.background
import pyopia.classify
import pyopia.instrument.silcam
import pyopia.instrument.holo
import pyopia.io
import pyopia.pipeline
import pyopia.plotting
import pyopia.process
import pyopia.statistics
import exampledata

import xarray
import matplotlib.pyplot as plt

## Load a TOML config file containing all setttings, and pipeline steps

This creates a dict of all settings

In [ ]:
toml_settings = pyopia.io.load_toml('config.toml')
toml_settings

## Make the steps dict

This is the dict that is given to `pyopia.pipeline.Pipeline`

The dict created here is not human-readable (but that shouldn't matter, since this came from the human-readable TOML config)

We are not including the disc writing in the pipeline for this example - so we can better demonstrate the formatted output data

In [ ]:
#steps = pyopia.pipeline.build_steps(toml_settings['steps'])
#steps

In [ ]:
processing_pipeline = pyopia.pipeline.Pipeline(toml_settings)

In [ ]:
raise

## Run the pipeline

(same as normal)

In [ ]:
# Initialise the pipeline and run the initial steps
processing_pipeline = pyopia.pipeline.Pipeline(toml_settings)

# Load an image (from the test suite)
filename = exampledata.get_example_silc_image()

# Process the image to obtain the stats dataframe
stats = processing_pipeline.run(filename)

## Write to disc

We will convert the STATS DataFrame into xarray for efficiency and easier metadata handling and easy writing to NetCDF.

This will use the steps part of the TOML configuration as metadata (attribute)

In [ ]:
pyopia.io.write_stats(
        'proc/test',
        stats,
        steps_string=toml_settings['steps'],
        append=False,
        format='nc')

## Load the data again

Then later, we can load the data again from NetCDF using xarray

xarray DataSets are presented nicely in notebooks

In [ ]:
xstats = xarray.open_dataset('proc/test-STATS.nc')
xstats

## Alter settings and re-process

What if we wanted to re-process this dataset with a different segmentation threshold?

### Get the TOML steps from the xarray DataSet

In [ ]:
toml_steps = pyopia.pipeline.steps_from_xstats(xstats)
toml_steps

### Alter the setting we want to change

In [ ]:
toml_steps['segmentation']['threshold'] = 0.9

### re-build the pipeline steps

In [ ]:
steps = pyopia.pipeline.build_steps(toml_steps)
steps

### re-process the pipeline

In [ ]:
# Initialise the pipeline and run the initial steps
processing_pipeline = pyopia.pipeline.Pipeline(steps)

# Load an image (from the test suite)
filename = exampledata.get_example_silc_image()

# Process the image to obtain the stats dataframe
stats = processing_pipeline.run(filename)

### Further analysis

At this point we could write this to disc again (like before)

and/or we could build a new, correctly formatted, xarray for immediate use (which we will do here):

In [ ]:
xstats_modified = pyopia.io.make_xstats(stats, toml_steps)
xstats_modified

### Plotting

We can plot directly from xarray in exactly the same way as from the Pandas DataFrame (so it doesn't matter which you use here). The benefit of 'xstats' as an xarray is that it now contains it's own metadata

In [ ]:
dias, vd = pyopia.statistics.vd_from_stats(xstats, 24)

plt.plot(dias, vd, label=f"Threshold={pyopia.pipeline.steps_from_xstats(xstats)['segmentation']['threshold']}")
plt.xscale('log')
plt.xlabel('ECD [um]')
plt.ylabel('Volume Distribution [uL/sample vol.]')

dias_modified, vd_modified = pyopia.statistics.vd_from_stats(xstats_modified, 24)

plt.plot(dias_modified, vd_modified, '--', label=f"Threshold={pyopia.pipeline.steps_from_xstats(xstats_modified)['segmentation']['threshold']}")

plt.legend()